In [ ]:
# load annotations and predictions
import os
import sys

import pandas as pd

# load Jamendo metadata
def load_jamendo_metadata():
    jamendo_meta = pd.read_csv("Lyric Sync/lyrics-alignment/jamendolyrics/JamendoLyrics.csv")
    return

pred_path = "data_pipeline/{lang}/predictions_{model}/{title}.csv"
ann_path = "data_pipeline/{lang}/annotations/{title}.words.csv"

In [5]:
jamendo_df = pd.read_csv("jamendolyrics/JamendoLyrics.csv")